In [2]:
import pandas as pd
import re
import jieba

In [3]:
df1 = pd.read_csv('sentiment_result_1.csv')
df2 = pd.read_csv('sentiment_result_2.csv')
df1

,微博id,发布者昵称,发布者性别,发布者地区,发布者关注数,发布者粉丝数,微博正文,发布时间,点赞数,转发数,评论数,predictions
0,J2Rk3u28b,新浪娱乐,女,北京,3381,40497222,新浪娱乐:【17-24时要闻回顾】俞灏明回应魏晨结婚时送房：原谅我当年的年少无知http:/...,2020-05-21 00:30:00,2815,169.0,218.0,1
1,J2Rj8bK6v,Cinderellapp月小昱,女,北京,99,1020,Cinderellapp月小昱:刚看了一集《探世界》 瞬间穿越回到旧金山 唐人街 干货店 粤...,2020-05-21 00:27:00,0,0.0,0.0,2
2,J2RfZ7cW0,有新欢如何成功挽回前男友,女,湖北,320,3228,有新欢如何成功挽回前男友:女生在提出分手的时候，很多人会说对你彻底失望了，我要分手但有些是内...,2020-05-21 00:20:00,0,0.0,0.0,5
3,J2VjWCmIo,科技阿康,男,北京,777,515083,科技阿康:#iOS13.5正式版推送#为了面对疫情，苹果正式推送了iOS13.5正式版的更新...,2020-05-21 10:40:00,38,13.0,40.0,2
4,J2V6HsCB7,泰娱乐论坛,男,北京,499,230234,泰娱乐论坛:#泰娱乐论坛热门泰剧# 【泰剧《倒霉女孩 / 珍贵的坏运气》人物剧照欣赏（第二部...,2020-05-21 10:08:00,13,0.0,2.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
56037,IwWEutkwn,搜狐新闻,男,北京,283,1359548,搜狐新闻:【华南海鲜市场展开大规模消杀工作 市场仍有一家四口在住】3月3日下午，包括武汉警方...,2020-03-03 22:28:00,333,15.0,65.0,0
56038,IwWEsb66i,Atm宁缺毋滥,女,黑龙江,419,27,Atm宁缺毋滥:疫情过后，国家领导们可否考虑一下，铁路人民也是一线人民？火车司机，出去三天三...,2020-03-03 22:28:00,0,0.0,0.0,6
56039,IwWEs7bhx,褚赛42,女,山东,4175,104680,褚赛42:【昆明一宠物店主动上交天竺鼠等52只动物】近日，民警在巡查眠山花鸟市场时，收到了市...,2020-03-03 22:28:00,0,0.0,0.0,4
56040,IwWErxCoj,徐州人民广播电台,男,江苏,1946,198963,徐州人民广播电台:【地铁2号线已全面复工！20座车站已全部封顶】人勤春来早，复工正当时。虽然...,2020-03-03 22:28:00,0,0.0,0.0,2


In [4]:
def remove_publisher_name(row):
    nickname = row['发布者昵称']
    content = row['微博正文']
    if content.startswith(nickname):
        return content[len(nickname):]
    return content

df1['微博正文'] = df1.apply(remove_publisher_name, axis=1)

In [5]:
def remove_non_chinese(content):
    return re.sub(r'[^\u4e00-\u9fa5]', '', content)

df1['微博正文'] = df1['微博正文'].apply(remove_non_chinese)

In [6]:
def segment_content(content):
    return jieba.lcut(content)

df1['分词微博正文'] = df1['微博正文'].apply(segment_content)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/cl/48s26tln7s52dn33yg1yx2f40000gn/T/jieba.cache
Loading model cost 0.350 seconds.
Prefix dict has been built successfully.


In [7]:
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())

additional_stopwords = {'原图', '展开', '全文', '显示', '地图', '组图', '共张', '图共张',
                        '图源', '图组', '超话', '图', '例', '说'}
stopwords.update(additional_stopwords)

The Chinese stop words are from: https://blog.csdn.net/AdamCY888/article/details/130025611?spm=1001.2101.3001.6650.4&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7ECTRLIST%7ERate-4-130025611-blog-120742737.235%5Ev43%5Econtrol&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7ECTRLIST%7ERate-4-130025611-blog-120742737.235%5Ev43%5Econtrol&utm_relevant_index=9

In [8]:
def remove_stopwords(segmented_content):
    return [word for word in segmented_content if word not in stopwords]

df1['分词微博正文'] = df1['分词微博正文'].apply(remove_stopwords)

In [9]:
df2['微博正文'] = df2.apply(remove_publisher_name, axis=1)
df2['微博正文'] = df2['微博正文'].apply(remove_non_chinese)
df2['分词微博正文'] = df2['微博正文'].apply(segment_content)
df2['分词微博正文'] = df2['分词微博正文'].apply(remove_stopwords)

In [10]:
df1.to_csv('word_cloud_df1.csv', index=False)
df2.to_csv('word_cloud_df2.csv', index=False)